In [1]:
from multiprocessing.pool import ThreadPool
import numpy as np
from pymoo.core.problem import ElementwiseProblem
from pymoo.core.problem import StarmapParallelization
from pymoo.constraints.as_obj import ConstraintsAsObjective
from pymoo.constraints.as_penalty import ConstraintsAsPenalty
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.operators.crossover.sbx import SBX
from pymoo.operators.mutation.pm import PM
from pymoo.operators.sampling.rnd import FloatRandomSampling
from pymoo.algorithms.soo.nonconvex.pso import PSO
from pymoo.optimize import minimize
from pymoo.termination import get_termination
import optimize as op
from cpg_fsm import *


In [2]:
L = 0.2
T_sw = 4
H_st = 0.2
freq = 100

class CorgiInitProblem(ElementwiseProblem):

    def __init__(self, **kwargs):
        super().__init__(n_var=4,
                         n_obj=1,
                         n_ieq_constr=24,
                         xl = np.array([-0.09, -0.09, -0.09, -0.09]),
                         xu = np.array([0.09, 0.09, 0.09, 0.09]),
                         **kwargs)

    def _evaluate(self, x, out, *args, **kwargs):
        FSM = FiniteStateMachine(freq)
        corgi = Corgi(freq, FSM)
        corgi.step_length = L
        corgi.stance_height = H_st
        corgi.swing_time = T_sw
        corgi.weight_s = 1
        corgi.weight_u = 0
        corgi.total_cycle = 4

        bp = np.array([ 0.04002311,  0.00303396,  0.01000181, -0.01986243,  0.01305118,
                        -0.03105092,  0.04006933,  0.00124093,  0.01742723, -0.04600008,
                        0.01166539, -0.02441404,  0.04008184,  0.00328895,  0.01020455,
                        -0.01692465,  0.01747139, -0.04729206,  0.04011618,  0.00043849,
                        0.01602226, -0.04007484,  0.01082458, -0.02170281])
        bp = bp.reshape(4,-1)

        
        tb_a = lk.InverseKinematicsPoly(np.array([[x[0]], [-H_st]]))
        tb_b = lk.InverseKinematicsPoly(np.array([[x[1]], [-H_st]]))
        tb_c = lk.InverseKinematicsPoly(np.array([[x[2]], [-H_st]]))
        tb_d = lk.InverseKinematicsPoly(np.array([[x[3]], [-H_st]]))
        corgi.setInitPhase(tb_a, tb_b, tb_c, tb_d)
        corgi.move(swing_profile=bp)
        f = corgi.cost

        out["F"] = [f]

# initialize the thread pool and create the runner
n_threads = 16
pool = ThreadPool(n_threads)
runner = StarmapParallelization(pool.starmap)
problem = CorgiInitProblem(elementwise_runner=runner, elementwise_evaluation=True)

In [3]:
algorithm = NSGA2(
    pop_size=16,
    n_offsprings=4,
    sampling=FloatRandomSampling(),
    crossover=SBX(prob=0.9, eta=15),
    mutation=PM(eta=20),
    eliminate_duplicates=True
)

termination = get_termination("n_gen", 100)

result = minimize(problem,
                  algorithm,
                  termination,
                  seed=1,
                  save_history=False,
                  verbose=False)

pool.close()

In [ ]:
print(result.X)
print(result.F)